In [3]:
%matplotlib inline

In [4]:
import glob
from astropy.table import Table, Column
from astropy.io import fits
from collections import Counter
from __future__ import absolute_import, division, print_function

import os
from collections import Counter
import numpy as np
from astropy.io import fits
from astropy.table import Table, Column

import astropy.constants
c = astropy.constants.c.to('km/s').value

from desitarget.targets import desi_mask
from desisim.quickcat import quickcat

### Looking for targets that were in fiberassign, but missing in zcat

In [10]:
tile_names = glob.glob("/home/forero/Data/desitest/lowfat_perfect/4/fiberassign/tile_*.fits")
nobs = Counter()
desitarget = np.empty((), dtype='int64')
targetid_fiber = np.empty((), dtype='int64')
for tile_file in tile_names:
    fibassign = Table.read(tile_file, hdu=1)
    ii = (fibassign['TARGETID'] != -1) 
    nobs.update(fibassign['TARGETID'][ii])
    desitarget = np.append(desitarget, np.array(fibassign['DESI_TARGET'][ii]))
    targetid_fiber = np.append(targetid_fiber, np.array(fibassign['TARGETID'][ii]))

print(len(nobs.keys()), len(targetid_fiber), len(desitarget))
print(targetid_fiber[:10])
targets = Table.read("/home/forero/Data/desitest/mtl/targets.fits")
truth = Table.read("/home/forero/Data/desitest/mtl/truth.fits")
mtl = Table.read("/home/forero/Data/desitest/lowfat_perfect/4/mtl.fits")
zcat = Table.read("/home/forero/Data/desitest/lowfat_perfect/4/zcat.fits")

570006 577718 577718
[           46095328 2848418669254187444 1615190646395193941
  719950519532907646  798326793562667321  966383442835711970
 3276782129000055637  170300333330830609 1463945693497080320
 3307573674961348008]


In [11]:
print(fibassign.colnames, len(targets['TARGETID']))

['FIBER', 'POSITIONER', 'NUMTARGET', 'PRIORITY', 'TARGETID', 'DESI_TARGET', 'BGS_TARGET', 'MWS_TARGET', 'RA', 'DEC', 'XFOCAL_DESIGN', 'YFOCAL_DESIGN'] 2947950


In [12]:
ids_fiber = list(nobs.keys())
ids_mtl = list(mtl['TARGETID'])
ids_truth = list(truth['TARGETID'])
ids_targets = list(targets['TARGETID'])
ids_zcat = list(zcat['TARGETID'])

In [7]:
def find_missing(list_A, list_B): # |list_A| > |list_B|
    a = list(set(list_A) & set(list_B)) #find the intersection
    missing = list(set(list_B) - set(a)) #find B minus the intersection
    return missing #these are the IDs in B that are missing in A

In [13]:
missing = find_missing(ids_targets, ids_mtl)
print(len(missing))
missing = find_missing(ids_targets, ids_fiber)
print(len(missing))
missing = find_missing(ids_mtl, ids_fiber)
print(len(missing))
missing = find_missing(list(set(targetid_fiber)), ids_fiber)
print(len(missing))
missing = find_missing(ids_truth, ids_fiber)
print(len(missing))
missing = find_missing(ids_zcat, ids_fiber)
print(len(missing))

0
65329
65329
0
65329
65329


The ```Counter(c[iiobs])``` tells us that in fact the missing targets are sky and standard stars. Nothing to worry.

In [16]:
a = np.array(targetid_fiber, dtype='int64')
b = np.array(missing)
c = np.array(desitarget, dtype='int64')
print(len(a), len(b), len(c))
iiobs = np.in1d( a, b)
#print(a, b)
print(Counter(c[iiobs]))
l  = nobs.keys()
l.sort()
b.sort()
print(l[-5:])
print(b[-5:])

577718 65329 577718
Counter({4294967296: 53473, 8589934592: 13369})
[4611631882328547593, 4611632229615896532, 4611647838028684250, 4611667287054992550, 4611679909359693146]
[4611496083463884945 4611522850556216644 4611550344386585230
 4611558383850489773 4611576400989880876]


### Checking that the number of observations is correctly propagated

(Based on S. Bailey code)

In [5]:
targets = Table.read("/home/forero/Data/desitest/mtl/targets.fits")
truth = Table.read("/home/forero/Data/desitest/mtl/truth.fits")

In [6]:
#- truth and targets are row-matched, so directly add columns instead of join
for colname in targets.colnames:
    if colname not in truth.colnames:
        truth[colname] = targets[colname]

In [8]:
print (truth.colnames)

['TARGETID', 'BRICKNAME', 'RA', 'DEC', 'TRUEZ', 'TRUETYPE', 'CATEGORY', 'DESI_TARGET', 'BGS_TARGET', 'MWS_TARGET', 'SUBPRIORITY']


In [9]:
Counter(zip(truth['DESI_TARGET'], truth['TRUETYPE']))

Counter({(1, 'GALAXY'): 291029,
         (1, 'UNKNOWN'): 48639,
         (2, 'GALAXY'): 2354756,
         (4, 'QSO'): 165428,
         (4, 'STAR'): 88098})

In [10]:
iilrg = ((truth['DESI_TARGET'] & desi_mask.LRG) != 0) & (truth['TRUETYPE'] == 'GALAXY')
iielg = ((truth['DESI_TARGET'] & desi_mask.ELG) != 0) & (truth['TRUETYPE'] == 'GALAXY')
iiqso = ((truth['DESI_TARGET'] & desi_mask.QSO) != 0) & (truth['TRUETYPE'] == 'QSO')

lrg = truth[iilrg]
elg = truth[iielg]
qso = truth[iiqso]

In [12]:
print('LRG', len(lrg))
print('ELG', len(elg))
print('QSO', len(qso))

LRG 291029
ELG 2354756
QSO 165428


In [14]:
zcat4 = Table.read('/home/forero/Data/desitest/lowfat_perfect/4/zcat.fits', 1)
zcat5 = Table.read('/home/forero/Data/desitest/lowfat_perfect/5/zcat.fits', 1)

In [15]:
def count_obs(zcat):
    obslrg = np.in1d(lrg['TARGETID'], zcat['TARGETID'])
    obselg = np.in1d(elg['TARGETID'], zcat['TARGETID'])
    obsqso = np.in1d(qso['TARGETID'], zcat['TARGETID'])
    print('LRG {:4f}'.format(np.count_nonzero(obslrg) / len(lrg)))
    print('ELG {:4f}'.format(np.count_nonzero(obselg) / len(elg)))
    print('QSO {:4f}'.format(np.count_nonzero(obsqso) / len(qso)))

In [18]:
print('zcat4')
count_obs(zcat4)
print('')
print('zcat5')
count_obs(zcat5)

zcat4
LRG 0.695501
ELG 0.524521
QSO 0.841321

zcat5
LRG 0.873868
ELG 0.678636
QSO 0.984700


In [ ]:
newzcat = quickcat(tile_names, targets, truth, perfect=True)

In [ ]:
targets_newzcat = list(newzcat['TARGETID'])
targets_newzcat = list(targets['TARGETID'])

In [ ]:
a = list(set(targets_newzcat) & set(targets_fiber))
missing_ids = list(set(targets_fiber) - set(a))
len(missing_ids)
print(missing_ids[:10])

In [ ]:
def myquickcat(tilefiles, targets, truth, zcat=None, perfect=False):
    """
    Generates quick output zcatalog
    
    Args:
        tilefiles : list of fiberassign tile files that were observed
        targets : astropy Table of targets
        truth : astropy Table of input truth with columns TARGETID, TRUEZ, and TRUETYPE
        
    Options:
        zcat : input zcatalog Table from previous observations
        perfect : if True, treat spectro pipeline as perfect with input=output,
            otherwise add noise and zwarn!=0 flags
        
    Returns:
        zcatalog astropy Table based upon input truth, plus ZERR, ZWARN,
        NUMOBS, and TYPE columns   
        
    TODO: Add BGS, MWS support     
    """
    #- convert to Table for easier manipulation
    truth = Table(truth)
    
    #- Count how many times each target was observed for this set of tiles
    ### print('Reading {} tiles'.format(len(obstiles)))
    nobs = Counter()
    for infile in tilefiles:
        fibassign = fits.getdata(infile, 'FIBER_ASSIGNMENTS')
        ii = (fibassign['TARGETID'] != -1)  #- targets with assignments
        nobs.update(fibassign['TARGETID'][ii])

        #- Count how many times each target was observed in previous zcatalog
    #- NOTE: assumes that no tiles have been repeated
    if zcat is not None:
        ### print('Counting targets from previous zobs')
        for targetid, n in zip(zcat['TARGETID'], zcat['NUMOBS']):
            nobs[targetid] += n #JF I put n instead of 1

    #- Trim truth down to just ones that have already been observed
    ### print('Trimming truth to just observed targets')
    obs_targetids = np.array(nobs.keys())
    truth_targetids = truth['TARGETID']
    
    a = list(set(truth_targetids) & set(nobs.keys()))
    missing_ids = list(set(nobs.keys()) - set(a))
    if len(missing_ids):
        print('missing ids {}'.format(len(missing_ids)))
        raise NameError('Some IDs in fiberassign are missing from the truth file')
   
    
    iiobs = np.in1d(truth['TARGETID'], obs_targetids)
    print('total {}, intersec {}'.format(len(nobs.keys()), len(truth[iiobs])))
    truth = truth[iiobs]
    targets = targets[iiobs]

    #- Construct initial new z catalog
    newzcat = Table()
    newzcat['TARGETID'] = truth['TARGETID']
    if 'BRICKNAME' in truth.dtype.names:
        newzcat['BRICKNAME'] = truth['BRICKNAME']
    else:
        newzcat['BRICKNAME'] = np.zeros(len(truth), dtype='S8')

    #- Copy TRUEZ -> Z so that we can add errors without altering original
    newzcat['Z'] = truth['TRUEZ'].copy()
    newzcat['TYPE'] = truth['TRUETYPE'].copy()

    #- Add numobs column
    ### print('Adding NUMOBS column')
    nz = len(newzcat)
    newzcat.add_column(Column(name='NUMOBS', length=nz, dtype=np.int32))
    for i in range(nz):
        newzcat['NUMOBS'][i] = nobs[newzcat['TARGETID'][i]]

    #- Add ZERR and ZWARN
    ### print('Adding ZERR and ZWARN')
    if not perfect:
        #- GALAXY -> ELG or LRG
        objtype = newzcat['TYPE'].copy()
        isLRG = (objtype == 'GALAXY') & ((targets['DESI_TARGET'] & desi_mask.LRG) != 0)
        isELG = (objtype == 'GALAXY') & ((targets['DESI_TARGET'] & desi_mask.ELG) != 0)
        objtype[isLRG] = 'LRG'
        objtype[isELG] = 'ELG'
        
        z, zerr, zwarn = get_observed_redshifts(objtype, newzcat['Z'])
        newzcat['Z'] = z  #- update with noisy redshift
    else:
        zerr = np.zeros(nz, dtype=np.float32)
        zwarn = np.zeros(nz, dtype=np.int32)

    newzcat['ZERR'] = zerr
    newzcat['ZWARN'] = zwarn

    #- Metadata for header
    newzcat.meta['EXTNAME'] = 'ZCATALOG'

    return newzcat



In [ ]:
mynewzcat = myquickcat(tile_names, targets, truth, perfect=True)

In [ ]:
targets_mynewzcat = list(mynewzcat['TARGETID'])
a = list(set(targets_mynewzcat) & set(targets_fiber))
missing_ids = list(set(targets_fiber) - set(a))
len(missing_ids)